# Publications Hypergraph Demo Notebook

This notebook is used to replcate the use case experiment in the paper **PAPER LINK HERE** that demonstrates the use of the hypergraph interchange formation (HIF).

The following hypergraph libraries have the HIF loading and saving functions allowing for the analysis of the same hypergraph over each library:
* HyperNetX (HNX)
* HyperGraphX (HGX)
* SimpleHypergraphs
* TopoNetX
* Hypergraph Analysis Toolbox (HAT)
* compleX Group Interactions (XGI)

## Publications Data as a Hypergraph

This dataset consists of open source publications with the keyword "Hypergraph" and was collected from ArXiv, Biorxiv, DBLP and Osti. The Hypergraph has hyperedges as publications and nodes as authors. The hyperedges have attributes including funding agencies, abstract, publication date, tags, and source, while the nodes have the attributes of institutions.

In [ ]:
# import standard packages for analysis and plotting
import json

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
file_path = "data/publications.hif.json"
with open(file_path, "r") as f:
    HIF_publications = json.load(f)

# HyperNetX

Here we demo loading in the hypergraph using HNX and demoing the unique homology capabilities of the library.

In [ ]:
import hypernetx as hnx

In [ ]:
H_hnx = hnx.from_hif(HIF_publications)

In [ ]:
hnx.draw(
    H_hnx,
    with_edge_labels=False,
    with_node_labels=False,
    node_radius=0.2,
    edges_kwargs={"lw": 0.5},
)

In [ ]:
# Getting the main connected component of hypergraph
import numpy as np

Hs = list(H_hnx.s_component_subgraphs(s=1, return_singletons=False))
I = np.argsort([len(H_CC.incidences.dataframe) for H_CC in Hs])
Hs = [Hs[i] for i in I]
H = Hs[-1]

In [ ]:
pos = hnx.draw(
    H,
    with_edge_labels=False,
    with_node_labels=True,
    node_radius=0.2,
    edges_kwargs={"lw": 0.5},
    return_pos=True,
)
plt.show()

## One-Dimensional Homology of a Hypergraph Using HNX

In [ ]:
import hypernetx as hnx
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

# Assuming your hypergraph 'H' is already defined
s = 1
edge_centrality = hnx.s_closeness_centrality(H, s=s)

# Get the centrality values and the edges
edges = list(H.edges)
centrality_values = np.array([edge_centrality[e] for e in edges])

# Normalize the centrality values to the range [0, 1]
if centrality_values.size > 0:
    normalized_centrality = (centrality_values - np.min(centrality_values)) / (
        np.max(centrality_values) - np.min(centrality_values)
    )
else:
    normalized_centrality = np.array([])  # Handle the case of no edges

# Choose a colormap
cmap = cm.get_cmap("viridis")

# Map the normalized centrality values to colors
edge_colors = cmap(normalized_centrality)

# Create a dictionary to pass to edges_kwargs for coloring
edge_style = {}
for i, edge in enumerate(edges):
    if i < len(edge_colors):
        edge_style[edge] = edge_colors[i]

# Create the hypergraph plot
fig, ax = plt.subplots(figsize=(12, 8))
hnx.draw(
    H,
    pos=pos,
    with_edge_labels=False,
    with_node_labels=True,
    node_radius=0.2,
    edges_kwargs={"color": list(edge_style.values()), "lw": 2, "alpha": 0.5},
    ax=ax,
)

# Add the colorbar
sm = cm.ScalarMappable(
    cmap=cmap,
    norm=plt.Normalize(vmin=np.min(centrality_values), vmax=np.max(centrality_values)),
)
sm.set_array([])  # For older versions of matplotlib
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label(f"{s}-Closeness Centrality")

plt.show()

In [ ]:
s = 1
print(f"The {s}-closeness centrality of the edges in hypergraph H is:")
print("-" * 100)

d = hnx.s_closeness_centrality(H, s=s)
for e in H.edges:
    print(f"{e}: {d[e]}")

# XGI

In [ ]:
import xgi

In [ ]:
H_xgi = xgi.read_hif(file_path)
testH = H_xgi.cleanup(in_place=False)

In [ ]:
fig = plt.figure(figsize=(5.5, 4))
ax = plt.gca()
# ax1 = ax.inset_axes([0, 0, 0.7, 1])
# ax1.set_axis_off()
xgi.draw(
    testH,
    # ax=ax1,
    pos=xgi.pca_transform(xgi.pairwise_spring_layout(testH, seed=0)),
    hull=True,
    node_size=testH.nodes.degree,
    node_fc=testH.nodes.clique_eigenvector_centrality,
)

ax1 = ax.inset_axes([0.7, 0.3, 0.25, 0.125])
ax1.plot(testH.nodes.degree.asnumpy(), "^", color="maroon", markersize=2, alpha=0.5, label="Self")
ax1.plot(testH.nodes.average_neighbor_degree.asnumpy(), "o", color="blue", markersize=2, alpha=0.5, label="Avg. neigh.")
ax1.set_ylabel("Degree")
sns.despine(ax=ax1)

ax2 = ax.inset_axes([0.7, 0.1, 0.25, 0.125])
sns.despine(ax=ax2)
ax2.plot(testH.nodes.clique_eigenvector_centrality.asnumpy(), "o", color="teal", markersize=2, alpha=0.5)
ax2.set_xlabel("Node")
ax2.set_ylabel("CEC")
plt.tight_layout()
plt.savefig("figures/xgi.svg", dpi=1000)

In [ ]:
plt.figure(figsize=(11, 4))
plt.subplot(121)
author = H_xgi.nodes.degree.argmax()

print(
    f"{author} has published the most papers in this dataset with {H_xgi.nodes.degree[author]}"
)
hist = H_xgi.nodes.degree.ashist(log_binning=True)
plt.scatter(hist["bin_center"], hist["value"])
hist = H_xgi.nodes.average_neighbor_degree.ashist(log_binning=True)
plt.scatter(hist["bin_center"], hist["value"])
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Papers co-authored")
plt.ylabel("Number of authors")
sns.despine()

plt.subplot(122)
paper = H_xgi.edges.size.argmax()
print(f"{paper} has the most co-authors with {H_xgi.edges.size[paper]}")
hist = H_xgi.edges.size.ashist(log_binning=True)
plt.scatter(hist["bin_center"], hist["value"])
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Number of co-authors")
plt.ylabel("Number of papers")
sns.despine()

# HAT

In [ ]:
from HAT import Hypergraph as HatHypergraph
from HAT.metrics import nonlinear_eigenvector_centrality

H_hat = HatHypergraph.from_hif(HIF_publications)
node_centrality, edge_centrality = nonlinear_eigenvector_centrality(H_hat, model='Linear')
i, e = max(enumerate(edge_centrality), key=lambda y: y[1])
name = H_hat.edges["edge"][i]
print(f'{name}: {e}')